# Dados Covid Estados Unidos

## Leitura/limpeza de dados

Os dados de casos/mortes por estado foram obtidos do [CDC (Center for Desease Control and Prevention)](https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36) dos Estados Unidos. Já os dados de população vem do [United States Census Bureau](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html).

In [44]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [45]:
covid_url = "../data/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"
census_url = "../data/nst-est2019-01.xlsx"

covid = pd.read_csv(covid_url,parse_dates = ['submission_date'])
covid = covid[['submission_date','state','new_death']]

census = pd.read_excel(census_url)

### Mortes por Covid

In [33]:
covid.head()

,submission_date,state,new_death
0,2021-01-25,NE,15
1,2020-01-25,OR,0
2,2020-07-23,KY,7
3,2020-04-01,VI,0
4,2020-09-04,NC,24


Mapeamento de siglas para nomes dos estados:

In [34]:
# Reference: https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))
covid = covid[covid.state.isin(abbrev_us_state.keys())]

In [35]:
def fill_na(state):
    filtered = vaccines[vaccines.location==state].sort_values(by='date')
    return filtered.fillna(method='ffill')

Não temos dados nulos, mas linhas com `new_death` negativo o que não faz sentido. Vamos imputar `NA` nessas linhas para que sejam ignorados nas funções de agregação por semana posteriormente.

In [36]:
covid[covid.new_death<0]

,submission_date,state,new_death
187,2020-10-18,ID,-1
199,2020-06-12,NV,-14
295,2020-04-28,MT,-1
387,2021-04-11,AZ,-6
604,2020-05-02,LA,-20
...,...,...,...
29363,2020-05-19,NV,-6
29788,2021-05-15,WA,-3
29882,2020-08-31,AZ,-1
29888,2020-12-10,WA,-166


In [37]:
negatives = list(covid[covid.new_death<0].index)
covid.loc[negatives,'new_death'] = np.nan
covid.loc[negatives,:]

,submission_date,state,new_death
187,2020-10-18,ID,NaN
199,2020-06-12,NV,NaN
295,2020-04-28,MT,NaN
387,2021-04-11,AZ,NaN
604,2020-05-02,LA,NaN
...,...,...,...
29363,2020-05-19,NV,NaN
29788,2021-05-15,WA,NaN
29882,2020-08-31,AZ,NaN
29888,2020-12-10,WA,NaN


Criaremos agora uma nova coluna para marcar a semana epidemiológica, considerando o domingo 01/03/2020 como semana 1.

In [38]:
covid.state = covid.state.apply(lambda x:abbrev_us_state[x])

In [39]:
covid = covid[covid.submission_date>=dt(2020,3,1)]
first_day = covid.sort_values(by='submission_date').iloc[1,0]
covid['week'] = (covid.submission_date - first_day)
covid.week = covid.week.apply(lambda x: x.days//7+1)
covid

,submission_date,state,new_death,week
0,2021-01-25,Nebraska,15.0,48
2,2020-07-23,Kentucky,7.0,21
4,2020-09-04,North Carolina,24.0,27
5,2021-04-11,New Jersey,11.0,59
7,2020-07-30,Maine,2.0,22
...,...,...,...,...
30112,2020-11-16,Rhode Island,6.0,38
30113,2021-05-30,Montana,0.0,66
30116,2020-10-11,Connecticut,0.0,33
30117,2021-04-26,Kentucky,3.0,61


### Dados Populacionais

Vamos agora limpar os dados do censo para obter uma tabela simples com a população dos 50 estados.

In [46]:
census.head()

,Unnamed: 0,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
1,Northeast,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
2,Midwest,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
3,South,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
4,West,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268


In [47]:
census.columns = census.columns[1:].insert(0,'state')
census.columns

Index(['state', 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype='object')

In [48]:
census = census[['state',2019]]
census.head(7)

,state,2019
0,United States,328239523
1,Northeast,55982803
2,Midwest,68329004
3,South,125580448
4,West,78347268
5,.Alabama,4903185
6,.Alaska,731545


In [50]:
census = census.iloc[5:,:]
census.head()

,state,2019
5,.Alabama,4903185
6,.Alaska,731545
7,.Arizona,7278717
8,.Arkansas,3017804
9,.California,39512223


In [51]:
remove_dot = lambda x:x[1:]
census.state = census.state.apply(remove_dot)
census = census[census.state.isin(us_state_abbrev.keys())]
census

,state,2019
5,Alabama,4903185
6,Alaska,731545
7,Arizona,7278717
8,Arkansas,3017804
9,California,39512223
10,Colorado,5758736
11,Connecticut,3565287
12,Delaware,973764
14,Florida,21477737
15,Georgia,10617423


Exportando os dados para csv:

In [12]:
covid.to_csv('data/covid_us_cleaned.csv')
census.to_csv('data/census_cleaned.csv')